In [ ]:
from cinnabar import plotting as cinnabar_plotting
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from cinnabar import FEMap
from rdkit import Chem

import math
import csv
import os

def find_exp_results(ligands_sdf_path: str, exp_property: str, output_path: str) -> None:
    """
    Find the experimental results from an SDF file
    
    Parameters
    ----------
    
    ligands_sdf_path: str
        Path to the SDF file containing the ligands and the experimental property.
    
    exp_property: str
        Name of the experimental property to find.
    
    output_path: str
        Path to the output file.
    """ 
    
    # Check if the file exists
    if not os.path.exists(ligands_sdf_path):
        raise FileNotFoundError(f"File {ligands_sdf_path} not found.")
    
    # Write header to output path
    with open(output_path, "w") as file:
        
        file.write("Ligand,exp_dG\n")
    
        # Load the SDF file
        suppl = Chem.SDMolSupplier(ligands_sdf_path, removeHs=False)
        
        for mol in suppl:
            
            if mol is None:
                print("Could not read molecule.")
                continue
            
            # Get the ligand name
            ligand_name = mol.GetProp("_Name")
            
            # Get the experimental property
            exp_value = mol.GetProp(exp_property)
            
            # Convert IC50 to dG (kJ/mol)
            if "IC50" in exp_property:
                # Assume the exp property is the IC50 in nM. dG = -RTln(IC50)
                exp_value = 8.314 * 298.15 * math.log(float(exp_value) * 1e-9) / 1000.0
                print(f"{ligand_name}:   IC50 {mol.GetProp(exp_property)} (nM)   dG {round(exp_value,3)} (kJ/mol)")
            elif "Kd" in exp_property:
                # Assume the exp property is the Kd in nM. dG = -RTln(Kd)
                exp_value = 8.314 * 298.15 * math.log(float(exp_value) * 1e-9) / 1000.0
                print(f"{ligand_name}:   Kd {mol.GetProp(exp_property)} (nM)   dG {round(exp_value,3)} (kJ/mol)")
            
            # Write the ligand name and experimental value to the output file
            file.write(f"{ligand_name},{exp_value}\n")
            
def show_edge_results(campaign_path: str, edge_name: str, plot_name: str, verbose: bool = False, solvent_too: bool = False):
    """
    Shows the specified plot for a given edge. It finds all the repetitions in the simulations folder and shows the images side by side.
    
    Parameters
    ----------
    
    campaign_results: str   
        Path to the campaign results folder.
    edge_name: str
        Name of the edge to show.
    plot_name: str
        Name of the plot to show.
    """
    
    def show_results_plot(paths: list, plot_name: str):
        """
        Shows the specified plot in the given paths. It shows the images side by side.
        
        Parameters
        ----------
        
        paths: list
            List of paths to the folders containing the results.
        plot_name: str
            Name of the plot to show.
        """
        
        imgs = []
        for path in paths:
             
            img_path = os.path.join(path, f"{plot_name}.png")
            
            if os.path.exists(img_path):
                imgs.append(mpimg.imread(img_path))
                
        if imgs:
            fig, axs = plt.subplots(1, len(imgs), figsize=(20, 10))
            for i, img in enumerate(imgs):
                axs[i].imshow(img)
                axs[i].axis('off')
            plt.show()
        else:
            if verbose:
                print(f"No results found for {plot_name}")
                print(f"Paths: {paths}")
    
    simulations_path = os.path.join(campaign_path, 'simulations')
    
    # Show results for transformation in the complex
    if verbose:
        print("Complex results")
    complex_path = os.path.join(simulations_path, f"{edge_name}_complex")
    complex_repeat_paths = [os.path.join(complex_path, x) for x in os.listdir(complex_path) if os.path.isdir(os.path.join(complex_path, x))]
    show_results_plot(complex_repeat_paths, plot_name)
    
    # Find results for transformation in the solvent
    if solvent_too:
        if verbose:
            print("Solvent results")
        solvent_path = os.path.join(simulations_path, f"{edge_name}_solvent")
        solvent_repeat_paths = [os.path.join(solvent_path, x) for x in os.listdir(solvent_path) if os.path.isdir(os.path.join(solvent_path, x))]
        show_results_plot(solvent_repeat_paths, plot_name)
    
def show_results(experimental_data: str, computational_data: str, convert_to_kcal: bool, target_name: str, title: str = "", 
                 plot_dGs: bool = True, draw_network: bool = False, plotly: bool = True):
    """
    Show computational vs experimental data using cinnabar.
    
    Parameters
    ----------
    
    experimental_data: str
        Path to the experimental data file.
    computational_data: str
        Path to the computational data file.
    convert_to_kcal: bool
        Convert experimental data to kcal/mol if in kJ/mol.
    target_name: str
        Name of the target protein.
    title: str
        Title of the plot.
    """
    
    conversion_factor = 1.0 / 4.184 if convert_to_kcal else 1.0
    
    # Load experimental data
    exp_data = {}
    with open(experimental_data) as file:
        rows = csv.reader(file, delimiter=',')
        next(rows)
        for row in rows:  
            exp_data[row[0]] = {'DG': float(row[1]) * conversion_factor}

    # Load computational data
    comp_data = {}
    with open(computational_data) as file:
        rows = csv.reader(file, delimiter='\t')
        next(rows)
        for row in rows:
            ddG_tag = f"{row[0]}->{row[1]}"
            comp_data[ddG_tag] = {
                'ligand_i': row[0],
                'ligand_j': row[1],
                'DDG': float(row[2]),
                'dDDG': float(row[3])
            }

    # Write a CSV compatible with Cinnabar
    cinnabar_input_path = 'cinnabar_input.csv'
    with open(cinnabar_input_path, 'w') as file:
        file.write("# Experimental block\n# Ligand, expt_DDG, expt_dDDG\n")
        for ligand, data in exp_data.items():
            file.write(f"{ligand}, {data['DG']:.2f}, 0.0\n")
        file.write("\n# Calculated block\n# Ligand1, Ligand2, calc_DDG, calc_dDDG(MBAR), calc_dDDG(additional)\n")
        for edge, data in comp_data.items():
            file.write(f"{data['ligand_i']}, {data['ligand_j']}, {data['DDG']:.2f}, 0, {data['dDDG']:.2f}\n")

    # Generate plots with Cinnabar
    fe = FEMap.from_csv(cinnabar_input_path)
    legacy_graph = fe.to_legacy_graph()
    
    if draw_network:
        fe.draw_graph()
        
    if plot_dGs:
        if plotly:
            cinnabar_plotting.plot_DDGs(legacy_graph, target_name=target_name, title=title, plotly=True, data_label_type='small-molecule')
            cinnabar_plotting.plot_DGs(legacy_graph, target_name=target_name, title=title, plotly=True, data_label_type='small-molecule')
        else:
            cinnabar_plotting.plot_DDGs(legacy_graph, target_name=target_name, title=title, plotly=False, data_label_type='small-molecule', figsize=10)
            cinnabar_plotting.plot_DGs(legacy_graph, target_name=target_name, title=title, plotly=False, data_label_type='small-molecule', figsize=10)
    
    return legacy_graph 

In [ ]:
import openfe
import konnektor
# from konnektor.visualization import draw_ligand_network
# from konnektor.visualization.visualization import draw_ligand_network
# from konnektor.visualization.widget import draw_network_widget
from openfe.utils.atommapping_network_plotting import plot_atommapping_network

ligand_network_path="Industry_benchmark2024/rbfe_network_planning/ligand_network.graphml" # New version changing names of ligands for clarity, see _bck for the original

with open(ligand_network_path) as f:
    graphml = f.read()

ligand_network = openfe.LigandNetwork.from_graphml(graphml)

fig = plot_atommapping_network(ligand_network)

# Save figure
fig.savefig("ligand_network.png", dpi=300, bbox_inches='tight')

In [ ]:
parent_folder='/path/to/parent/folder'
exp_data = os.path.join(parent_folder, 'exp_results.tsv')
ligands_sdf_path = 'input/ligands_with_exp_filtered.sdf' 

find_exp_results(ligands_sdf_path, 'r_user_Kd', exp_data)

protocol_path = os.path.join(parent_folder, 'Industry_benchmark2024')
comp_data = os.path.join(protocol_path, 'ddG_results.tsv')

graph = show_results(experimental_data=exp_data, computational_data=comp_data, convert_to_kcal=True, target_name='DHODH', title = "1 ns equil - 5 ns prod", plotly=False)

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw

# Extract the contents of the sdf file and visualise it
ligands_rdmol = [mol for mol in
                 Chem.SDMolSupplier(ligands_sdf_path, removeHs=False)]

for ligand in ligands_rdmol:
    AllChem.Compute2DCoords(ligand)

# Draw the ligands with the names
Chem.Draw.MolsToGridImage(ligands_rdmol, molsPerRow=4, subImgSize=(200, 200), legends=[mol.GetProp("_Name") for mol in ligands_rdmol])